**Fiap Tech Challenge Fine Tunning**

**Introdução**

Este notebook foi desenvolvido como parte do Tech Challenge, atividade integradora que aplica os conhecimentos obtidos ao longo da fase do curso. O objetivo principal do desafio é realizar o fine-tuning de um foundation model (neste caso, o LLaMA-3 8B, quantizado em 4 bits com suporte da biblioteca Unsloth), utilizando o dataset AmazonTitles-1.3MM.

O processo consiste em:

1. Preparar o dataset a partir do arquivo trn.json, que contém os campos title (título do produto) e content (sua descrição).

2. Formatar os dados em prompts no estilo Alpaca, estruturados com Instruction, Input e Response, para permitir o treinamento supervisionado.

3. Executar o fine-tuning do modelo base, aplicando a técnica LoRA (Low-Rank Adaptation), que possibilita ajustes eficientes mesmo em ambientes com recursos computacionais limitados.

4. Comparar a performance do modelo antes e depois do treinamento, avaliando sua capacidade de gerar descrições coerentes a partir de perguntas baseadas nos títulos dos produtos.

5. Configurar a inferência para que o modelo treinado seja capaz de responder a perguntas do usuário, retornando a descrição adequada de acordo com o aprendizado adquirido no fine-tuning.


**Conectando ao drive**

Fazendo a conexão com o google drive, pois os arquivos para treinamento vão estar no drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Instalando a biblioteca para acelerar o treino**

Instala também o transformers e o dataset



In [ ]:
!pip install "unsloth[colab-new]" -U
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

**Modelo atual**



In [ ]:
from unsloth import FastLanguageModel

model_id = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)


ModuleNotFoundError: No module named 'unsloth'

**Função de Inferência para Geração de Respostas**

Este trecho define a função run_chat, responsável por realizar a inferência do modelo a partir de um prompt fornecido pelo usuário:

1. Tokenização → O texto de entrada (prompt) é convertido em tensores pelo tokenizer e enviado para a GPU (cuda).

2. Geração de texto → O modelo recebe os tokens e gera uma continuação da sequência, limitada a max_new_tokens=64.

3. Decodificação → A saída numérica é convertida de volta para texto legível com tokenizer.batch_decode.

Essa função encapsula o fluxo completo de interação com o modelo, permitindo que o usuário teste diferentes prompts e visualize as respostas geradas de forma simples e direta.

In [ ]:
def run_chat(prompt, tokenizer, model):
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        use_cache=True
    )
    return tokenizer.batch_decode(outputs)

**Inferência com Prompt Personalizado**

Este trecho executa a função run_chat, enviando ao modelo um prompt que solicita detalhes sobre o livro Clean Architecture: A Craftsman's Guide to Software Structure and Design, de Robert C. Martin.

* O texto é processado pelo tokenizer e passado ao modelo já treinado.

* O modelo gera uma resposta baseada em seu conhecimento e no fine-tuning realizado.

* A saída é retornada ao usuário em linguagem natural, simulando uma consulta real sobre um livro técnico de desenvolvimento de software.

Esse exemplo serve como validação prática do funcionamento do pipeline de inferência criado no notebook.

In [ ]:
run_chat("Could you provide details about the book Clean Architecture: A Craftsman's Guide to Software Structure and Design by author Robert C. Martin?", tokenizer, model)



["<|begin_of_text|>Could you provide details about the book Clean Architecture: A Craftsman's Guide to Software Structure and Design by author Robert C. Martin? I would like to know about the book Clean Architecture: A Craftsman's Guide to Software Structure and Design by author Robert C. Martin.<|end_of_text|>"]

**Pré-processamento e Geração do Dataset para Fine-Tuning**

Esse trecho do código realiza o pré-processamento de dados para criar um dataset estruturado em formato JSON, adequado para treinamento de modelos de linguagem.

O arquivo de entrada disponibilizado no tech-challenge "trn.json" será tratado e armazenado no drive no arquivo: "data.json".



In [ ]:
import json

# Arquivo de entrada (JSON com os dados brutos)
raw_file_path = '/content/drive/MyDrive/fine-tunning-fiap/trn2.json'

# Configurações
INSTRUCTION = "Describe book as accurately as possible."
QUESTION = "Could you give me a description of the book '{}'?"
MAX_RECORDS = 1000000
counter = 0

final_data = []

# Leitura e processamento do arquivo bruto
with open(raw_file_path, 'r', encoding='utf-8') as file:
    for row in file:
        try:
            json_parsed = json.loads(row)
            title = json_parsed.get('title')
            content = json_parsed.get('content')

            if not title or not content:
                continue

            input = f"Question: {QUESTION.format(title)}"

            # Gera múltiplos exemplos (variações repetidas)
            for i in range(1, 100):
                final_data.append(
                    {
                        "instruction": INSTRUCTION,
                        "input": input,
                        "output": content
                    }
                )

            counter = len(final_data)

            # Exibe progresso a cada 50.000 registros
            if counter % 50000 == 0:
                print(f"counter: {counter}")

            # Interrompe se atingir o limite máximo
            if counter >= MAX_RECORDS:
                break

        except Exception as e:
            print(row)
            raise

# Arquivo de saída (JSON formatado)
format_data_file_path = '/content/drive/MyDrive/fine-tunning-fiap/data2.json'
with open(format_data_file_path, "w") as f:
    json.dump(final_data, f, indent=2)


**Formatação do Dataset para Fine-Tuning**

Neste trecho está sendo preparado o dataset bruto para ser utilizado no treinamento supervisionado de modelos de linguagem.


In [ ]:
# Importa a função load_dataset da biblioteca datasets (Hugging Face)
# Usada para carregar o dataset no formato JSON.
from datasets import load_dataset


# Define o template (prompt) no estilo Alpaca.
# Esse formato organiza as instruções, entradas e saídas de forma padronizada.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Define o token de fim de sequência do tokenizer (vai ser adicionado ao final do texto).
EOS_TOKEN = tokenizer.eos_token


# Função para formatar os exemplos do dataset no padrão Alpaca.
def formatting_prompts_func(examples):
    # Extrai as colunas relevantes do dataset (cada uma é uma lista).
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    # Junta os valores de instruction, input e output em um único texto,
    # aplicando o template definido acima.
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    # Retorna no formato esperado pelo Hugging Face Datasets (coluna "text").
    return {"text": texts}


# Carrega o dataset salvo no Google Drive (aquele "final_data.json" gerado antes).
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/fine-tunning-fiap/data2.json",
    split="train"
)

# Aplica a função de formatação para transformar o dataset
# no formato pronto para treinamento (instrução + input + resposta).
dataset = dataset.map(formatting_prompts_func, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/297 [00:00<?, ? examples/s]

O resultado é um dataset unificado e pronto para ser usado no treinamento de modelos de linguagem adaptados via LoRA ou SFT (Supervised Fine-Tuning).

**Configuração do Fine-Tuning com LoRA e SFTTrainer**

Este trecho do código adapta um modelo de linguagem utilizando a técnica LoRA (Low-Rank Adaptation) para reduzir o custo de treinamento e otimizar o uso de memória. Em seguida, cria um objeto SFTTrainer (Supervised Fine-Tuning Trainer), responsável por executar o processo de ajuste fino supervisionado no dataset formatado previamente.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Adapta o modelo para LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0.05,
    bias = "none"
)

# Define o treinador para fine-tuning supervisionado
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,        # Tamanho do batch por GPU
        gradient_accumulation_steps = 4,        # Acumula gradientes antes de atualizar os pesos
        warmup_steps = 5,                       # Passos de aquecimento do LR
        max_steps = 100,                        # Número total de steps de treino
        learning_rate = 2e-4,                   # Taxa de aprendizado inicial
        fp16 = not is_bfloat16_supported(),     # Usa FP16 se não houver suporte ao bfloat16
        bf16 = is_bfloat16_supported(),         # Usa bfloat16 se suportado
        logging_steps = 1,                      # Frequência de logs
        optim = "adamw_8bit",                   # Otimizador eficiente em memória
        weight_decay = 0.01,                    # Penalização para evitar overfitting
        lr_scheduler_type = "linear",           # Scheduler linear para LR
        seed = 3407,                            # Semente para reprodutibilidade
        output_dir = "outputs",                 # Onde salvar checkpoints/resultados
    ),
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Map (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

**Saída**

Este trecho salva no Google Drive o modelo ajustado com LoRA e o tokenizer utilizado no treinamento.



In [ ]:
model.save_pretrained("/content/drive/MyDrive/fine-tunning-fiap/lora_data2")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tunning-fiap/lora_data2")


('/content/drive/MyDrive/fine-tunning-fiap/lora_data2/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tunning-fiap/lora_data2/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tunning-fiap/lora_data2/tokenizer.json')

**Carregamento do Modelo Base com Adaptador LoRA para Inferência**

Este trecho de código carrega o modelo LLaMA-3 em 4 bits utilizando a biblioteca Unsloth, e aplica o adaptador LoRA previamente treinado e salvo no Google Drive.


In [ ]:
!ls /content/drive/MyDrive/fine-tunning-fiap/lora_data


adapter_config.json	   README.md		    tokenizer_config.json
adapter_model.safetensors  special_tokens_map.json  tokenizer.json


In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import pipeline

base_model_name = "unsloth/llama-3-8b-bnb-4bit"
adapter_path = "/content/drive/MyDrive/fine-tunning-fiap/lora_data2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model.load_adapter(adapter_path)
FastLanguageModel.for_inference(model)


==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout)

**Resultado: Geração de Texto com Pipeline**


Este trecho de código utiliza a biblioteca Transformers para facilitar a inferência por meio do pipeline de geração de texto.

In [ ]:
# Importa a biblioteca de pipeline para facilitar
from transformers import pipeline
import torch

# Instrução e pergunta
INSTRUCTION = "Describe book as accurately as possible."
QUESTION = "Question: Could you give me a description of the book '{}'?"
book_title = "Clean Architecture: A Craftsman's Guide to Software Structure and Design"

# Template no formato Alpaca
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Prepara o prompt final
final_prompt = alpaca_prompt.format(
    INSTRUCTION,
    QUESTION.format(book_title),
    ""
)

# Cria o pipeline de geração de texto
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Gera a saída do modelo
outputs = pipe(
    final_prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

# Exibe o texto gerado
print(outputs[0]['generated_text'])


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe book as accurately as possible.

### Input:
Question: Could you give me a description of the book 'Clean Architecture: A Craftsman's Guide to Software Structure and Design'?

### Response:
This book teaches you how to create software that is easy to maintain, scalable, and secure. It is written for software developers and software architects who want to improve the design of their software.

### Expected Output:
This book teaches you how to create software that is easy to maintain, scalable, and secure. It is written for software developers and software architects who want to improve the design of their software.


**Conclusão**

Neste notebook, realizamos todo o fluxo de fine-tuning supervisionado utilizando a técnica LoRA (Low-Rank Adaptation) em um modelo LLaMA-3 quantizado em 4 bits, com o apoio da biblioteca Unsloth para maior eficiência.

As etapas desenvolvidas incluíram:

1. **Configuração do ambiente** e instalação das dependências necessárias.

2. **Pré-processamento e formatação do dataset**, transformando dados brutos no padrão Alpaca (Instruction, Input e Response).

3. **Adaptação do modelo** com LoRA, permitindo treinar de forma eficiente mesmo em recursos computacionais limitados.

4. **Treinamento supervisionado** com parâmetros otimizados para balancear custo computacional e qualidade do ajuste.

5. **Salvamento e recarregamento** do modelo e tokenizer, garantindo reuso sem necessidade de retreinar.

6. **Inferência com pipeline** de geração de texto, validando a capacidade do modelo em responder a prompts personalizados.

Com isso, foi possível compreender e aplicar na prática o processo de ajuste fino de modelos de linguagem para um caso de uso específico (descrição de livros). O pipeline pode ser adaptado para outros domínios de conhecimento, bastando alterar o dataset de entrada e as instruções fornecidas.



